In [40]:
!pip install pysentimiento

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 37.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.w

In [78]:
import os
import ast
import warnings
import matplotlib
import numpy as np
import pandas as pd
from sklearn.svm import SVC
import statsmodels.api as sm
from google.colab import drive
import matplotlib.pyplot as plt
from collections import Counter
warnings.filterwarnings("ignore")
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from pysentimiento import create_analyzer
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
drive.mount('/content/drive', force_remount=True)
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from pysentimiento.preprocessing import preprocess_tweet
from statsmodels.tsa.stattools import grangercausalitytests
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_selection import SelectKBest, mutual_info_classif, f_classif, chi_2
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, BaggingClassifier
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split, GridSearchCV

Mounted at /content/drive


In [7]:
dataset_path = os.path.join(os.getcwd(), "drive/My Drive/BDS_proj/data")
raw = os.path.join(dataset_path, "raw.csv")
trend = os.path.join(dataset_path, "google_trend.csv")

In [10]:
df = pd.read_csv(raw)

,id,title,budget,release_date,revenue,runtime,vote_average,vote_count,popularity,genres,production_companies,overview,year,month,actors,core_actor,actor_avg_age
0,940551,Migration,72000000,2023/12/6,295556285,83,7.500,1203,853.910,"['Animation', 'Action', 'Adventure', 'Comedy',...",[6704],After a migrating duck family alights on their...,2023,12,"['Kumail Nanjiani', 'Tresi Gazal', 'Elizabeth ...",50.0,32.0
1,787699,Wonka,125000000,2023/12/6,632302312,117,7.193,2852,400.772,"['Comedy', 'Family', 'Fantasy']","[174, 79, 214904, 437, 216687]",Willy Wonka _ chock-full of ideas and determin...,2023,12,"['Timothée Chalamet', 'Gustave Die', 'Murray M...",120.0,28.0
2,1072790,Anyone But You,25000000,2023/12/21,214891370,104,7.073,1349,375.097,"['Romance', 'Comedy']","[7291, 105052, 124283, 5]","After an amazing first date, Bea and Ben__ fie...",2023,12,"['Sydney Sweeney', 'Glen Powell', 'Alexandra S...",258.0,31.0
3,792307,Poor Things,35000000,2023/12/7,114912799,142,7.809,3131,356.631,"['Science Fiction', 'Romance', 'Comedy']","[127929, 6705, 22213, 3353, 72597, 138287]",Brought back to life by an unorthodox scientis...,2023,12,"['Emma Stone', 'Mark Ruffalo', 'Willem Dafoe']",89.0,53.0
4,572802,Aquaman and the Lost Kingdom,205000000,2023/12/20,434381226,124,6.793,2271,273.622,"['Action', 'Adventure', 'Fantasy']","[174, 11565, 76907, 128064, 216687]",Black Manta seeks revenge on Aquaman for his f...,2023,12,"['Jason Momoa', 'Patrick Wilson', 'Yahya Abdul...",92.0,43.0


In [67]:
dataset_path = os.path.join(os.getcwd(), "drive/My Drive/BDS_proj/data")
raw = os.path.join(dataset_path, "raw.csv")
trend = os.path.join(dataset_path, "google_trend.csv")
df = pd.read_csv(raw)
df = df[["overview", "budget", "revenue", "runtime", "year", "release_date", "genres", "vote_average", "popularity", "core_actor", "actor_avg_age", "production_companies", "month"]]
df["budget"] = df["budget"].astype(int)
df["runtime"] = df["runtime"].astype(int)
df = df[df["core_actor"].notna()]
df = df[df["actor_avg_age"] > 10]
df.dropna(inplace=True)
df["profitable"] = df["revenue"] / df["budget"]
df["profitable"] = df["profitable"] > 1
df["profitable"] = df["profitable"].astype(int)
df = df.drop(["revenue"], axis=1)

all_genres = set()
genres_list = df['genres'].apply(ast.literal_eval)
for genre_list in genres_list:
    for genre in genre_list:
        all_genres.add(genre)
all_genres = list(all_genres)
df[all_genres] = 0
for i, genre_list in enumerate(genres_list):
    df.loc[i, genre_list] = 1
df = df.drop(["genres"], axis=1)
df = df.dropna()
company_freq = {}
df['production_companies'] = df['production_companies'].apply(ast.literal_eval)
for company_list in df["production_companies"]:
    for company_id in company_list:
        if company_id not in company_freq:
            company_freq[company_id] = 0
        company_freq[company_id] += 1
company_freq = {x:y for x, y in company_freq.items() if y > 6}
df["company_freq"] = 0
for i, company_list in enumerate(df["production_companies"]):
    for id in company_list:
        if id in company_freq:
            df.loc[i, ["company_freq"]] = 1
            break
df = df.drop(["production_companies"], axis=1)
df = df.dropna()
df['release_date'] = df['release_date'].astype('datetime64[ns]')

trend = pd.read_csv(trend)
trend["begin_date"] = pd.to_datetime(trend["begin_date"])
trend["end_date"] = trend["begin_date"] + pd.DateOffset(days=7)
trend["movie_count"] = 0
trend["movie_gain"] = 0

df["google_trend"] = 0
for i, date in enumerate(df["release_date"]):
    interest = trend[(trend["begin_date"] <= date) & (date <= trend["end_date"])]["interest"].iloc[0]
    df.loc[i, ["google_trend"]] = interest

    trend.loc[(trend["begin_date"] <= date) & (date <= trend["end_date"]), ["movie_count"]] += 1
    if df.loc[i, ["profitable"]].iloc[0] == 1:
        trend.loc[(trend["begin_date"] <= date) & (date <= trend["end_date"]), ["movie_gain"]] += 1
df = df.drop(["release_date"], axis=1)
trend['cumulative_gain'] = trend['movie_gain'].cumsum()
trend['cumulative_count'] = trend['movie_count'].cumsum()
df = df.reset_index(drop=True).dropna()
emotion_analyzer = create_analyzer(task="emotion", lang="en")
df["emotion"] = df.overview.apply(lambda x:emotion_analyzer.predict(x).probas)
for i in df.emotion[0].keys():
  df[i] = df.emotion.apply(lambda x: x[i] if pd.notnull(x) else x)
df.drop(columns = ['overview', 'emotion'],inplace=True)


In [90]:
X = df.drop(columns=["profitable"])
y = df.profitable
x = X.values
min_max_scaler = MinMaxScaler()
X_scaled = pd.DataFrame(min_max_scaler.fit_transform(x), columns = X.columns)
selector = SelectKBest(f_classif, k=10)
selector.fit(X_scaled, y)
cols_idxs = selector.get_support(indices=True)
X_new = X_scaled.iloc[:,cols_idxs]
X_new.dtypes

budget          float64
runtime         float64
year            float64
vote_average    float64
popularity      float64
Comedy          float64
google_trend    float64
others          float64
anger           float64
disgust         float64
dtype: object

In [ ]:
def compare_scores(scores_dict):
    df = pd.DataFrame(scores_dict).T.applymap(np.mean).style.background_gradient(cmap='RdYlGn')
    return df



X = X_new
(
  X_train,
  X_test,
  y_train,
  y_test,
) = train_test_split(X, y, test_size=0.3)
classifiers = {'KNN': KNeighborsClassifier(),
              'Naive Bayes': GaussianNB(),
              'Decision Tree': DecisionTreeClassifier(),
              'Neural Network': MLPClassifier(),
              'Support Vector': SVC(),
              'Stochastic Gradient Descent': SGDClassifier(),
              'Random Forest': RandomForestClassifier(),
              'Gradient Boosting': GradientBoostingClassifier(),
              'Bagging': BaggingClassifier(KNeighborsClassifier())}
scores_train = {}
for classifier in classifiers:
    scores_train[classifier] = cross_validate(classifiers[classifier],
            X_train, y_train, cv=10, scoring=[
        'accuracy',
        'precision',
        'recall',
        'f1',
        'f1_macro',
        'f1_micro',
        ])
compare_scores(scores_train)

In [83]:
clf1 = SVC()
clf2 = MLPClassifier(max_iter=100)
clf3 = RandomForestClassifier()

param1 = {}
param1['classifier__kernel'] = ['linear', 'rbf']
param1['classifier__gamma'] = [0.1, 1, 0.01, 0.0001]
param1['classifier__C'] = [0.1, 1, 10, 100]
param1['classifier__degree'] = [0, 1, 2, 3, 4, 5]
param1['classifier'] = [clf1]
param2 = {}
param2['classifier__hidden_layer_sizes'] = [(50,), (10,), (100,)]
param2['classifier__activation'] = ['tanh', 'relu']
param2['classifier__solver'] = ['sgd', 'adam','lbfgs']
param2['classifier__alpha'] = [0.0001, 0.001, 0.01, 0.05]
param2['classifier__learning_rate'] = ['constant','adaptive']
param2['classifier'] = [clf2]
param3 = {}
param3['classifier__n_estimators'] = [5, 10, 50, 100]
param3['classifier__max_depth'] = [None, 5, 10, 15, 20]
param3['classifier__max_features'] = [3, 5 ,7]
param3['classifier__min_samples_leaf'] = [7, 3, 5]
param3['classifier__min_samples_split'] = [7, 3, 5]
param3['classifier'] = [clf3]
pipeline = Pipeline([('classifier', clf1)])
params = [param1, param2, param3]
gs = GridSearchCV(pipeline, params, scoring='f1').fit(X_train,
        y_train)
print('Best parameter set: ' + str(gs.best_params_))
print('Best f1: %.3f' % gs.best_score_)

Best parameter set: {'classifier': RandomForestClassifier(max_depth=15, max_features=3, min_samples_leaf=5,
                       min_samples_split=5, n_estimators=50), 'classifier__max_depth': 15, 'classifier__max_features': 3, 'classifier__min_samples_leaf': 5, 'classifier__min_samples_split': 5, 'classifier__n_estimators': 50}
Best f1: 0.792


In [84]:
predic = gs.predict(X_test)
print(classification_report(y_test,predic))
print(confusion_matrix(y_test, predic))

              precision    recall  f1-score   support

         0.0       0.60      0.71      0.65        58
         1.0       0.69      0.58      0.63        64

    accuracy                           0.64       122
   macro avg       0.64      0.64      0.64       122
weighted avg       0.65      0.64      0.64       122

[[41 17]
 [27 37]]


In [ ]:
data = pd.read_csv(bin_data)[['profitable','google_trend']]
test='ssr_chi2test'
variables = data.columns
verbose = False
df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
for c in df.columns:
    for r in df.index:
        test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
        p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
        if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
        min_p_value = np.min(p_values)
        df.loc[r, c] = min_p_value
df.columns = [var + '_x' for var in variables]
df.index = [var + '_y' for var in variables]

In [ ]:
df

,profitable_x,google_trend_x
profitable_y,1.0000,0.0022
google_trend_y,0.1681,1.0000
